# 常见问题与解答

## 如何引用Keras？

如果Keras对你的研究有帮助的话，请在你的文章中引用Keras。这里是一个使用BibTex的例子：
```
@misc{chollet2015keras,
  author = {Chollet, François and others},
  title = {Keras},
  year = {2015},
  publisher = {GitHub},
  journal = {GitHub repository},
  howpublished = {\url{https://github.com/fchollet/keras}}
}
```

## 如何使Keras调用GPU？

如果采用Tensorflow作为后端，当机器上有可用的GPU时，代码会自动调用GPU进行并行计算。如果使用Theano作为后端，可以通过以下方法设置：

方法1：使用Theano标记

在执行Python脚本时使用下面的命令：
``` shell
THEANO_FLAGS=device=gpu,floatX=float32 python my_keras_script.py
```

方法2：设置.theanorc文件

点击[这里](http://deeplearning.net/software/theano/library/config.html)查看指导教程。

方法3：在代码的开头手动设置theano.config.device和theano.config.floatX：

``` python
import theano
theano.config.device = 'gpu'
theano.config.floatX = 'float32'
```

## 如何在多张GPU卡上使用Keras？

我们建议有多张GPU卡可用时，使用Tensorflow后端。

有两种方法可以在多张GPU上运行一个模型：数据并行/设备并行。

大多数情况下，你需要的很可能是“数据并行”。

### 数据并行

数据并行将目标模型在多个设备上各复制一份，并使用每个设备上的复制品处理整个数据集的不同部分数据。Keras在keras.util.multi_gpu_model中提供有内置函数，该函数可以产生任意模型的数据并行版本，最高支持在8片GPU上并行。下面是一个例子：
``` python
from keras.utils import multi_gpu_model

# Replicates `model` on 8 GPUs.
# This assumes that your machine has 8 available GPUs.
parallel_model = multi_gpu_model(model, gpus=8)
parallel_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# This `fit` call will be distributed on 8 GPUs.
# Since the batch size is 256, each GPU will process 32 samples.
parallel_model.fit(x, y, epochs=20, batch_size=256)
```

### 设备并行

设备并行是在不同设备上运行同一个模型的不同部分，当模型含有多个并行结构，例如含有两个分支时，这种方式很适合。

这种并行方法可以通过使用Tensorflow device scopes实现，下面是一个例子：
``` python
# Model where a shared LSTM is used to encode two different sequences in parallel.
tweet_a = keras.Input(shape=(140, 256))
tweet_b = keras.Input(shape=(140, 256))

shared_lstm = keras.layers.LSTM(64)

# Process the first sequence on one GPU
with tf.device_scope('/gpu:0'):
    encoded_a = shared_lstm(tweet_a)
# Process the next sequence on another GPU
with tf.device_scope('/gpu:1'):
    encoded_b = shared_lstm(tweet_b)

# Concatenate results on CPU
with tf.device_scope('/cpu:0'):
    merged_vector = keras.layers.concatenate([encoded_a, encoded_b], axis=-1)
```

## "batch"，"epoch"和"sample"都是什么意思？

下面是一些使用Keras时常会遇到的概念，我们来简单解释。
- sample：样本，数据集中的一条数据。例如图片数据集中的一张图片，语音数据中的一段音频。
- batch：中文为批，一个batch由若干条数据构成。batch是进行网络优化的基本单位，网络参数的每一轮优化需要使用一个batch。batch中的样本是被并行处理的。与单个样本相比，一个batch的数据能更好的模拟数据集的分布，batch越大则对输入数据分布模拟的越好，反映在网络训练上，则体现为能让网络训练的方向“更加正确”。但另一方面，一个batch也只能让网络的参数更新一次，因此网络参数的迭代会较慢。在测试网络的时候，应该在条件的允许范围内尽量使用更大的batch，这样计算效率会更高。
- epoch：epoch可译为“轮次”。如果说每个batch对应网络的一次更新的话，一个epoch对应的就是网络的一轮更新。每一轮更新中网络的更新次数可以随意，但通常会设置为遍历一遍数据集。因此一个epoch的含义是模型完整的看了一遍数据集。设置epoch的主要作用是把模型的训练的整个训练过程分为若干段，这样我们可以更好的观察和调整模型的训练。Keras中，当指定了验证集时，每个epoch执行完后都会运行一次验证集以确定模型的性能。另外，我们可以使用回调函数在每个epoch的训练前后执行一些操作，如调整学习率，打印目前模型的一些信息等，详情请参考callback一节。

## 如何保存Keras模型？

